#### Code to put correct Tmb, FWMH and Tkin into radex and store results of column density.

In [1]:
# gaussian fits for all pixels in an image

#import astropy.units as u
from astropy.utils import data

import numpy as np
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from astropy.io import fits
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.modeling import models, fitting
from astropy.io import ascii
from astropy.table import Table


import warnings
warnings.filterwarnings('ignore') # Turn warnings off (because they're annoyong!)

# Change the parameters below for each new source
################################################

path='/Users/jane/Desktop'   # path to file

filename='s138_HCO+_smo.fits'       # input fits file name
outfile = 's138_HCO+_gaussfitsTEST.txt' # file name of the gaussian fit results

vel_guess = -52  # The initial guess for the centroid velocity of the gaussian fit
dv_min = 1       # set a minimum acceptable line width
dv_max = 9       # set a maximum acceptable line width
snr_min = 2.5    # set a minumum acceptable SNR

# set the range of pixels in the x direction over which to loop
min_xpix = 3
max_xpix = 10

# set the range of pixels in the y direction over which to loop
min_ypix = 39
max_ypix = 46

# set the velocity range over which to calculate the line integrated intensity
min_vline = -56
max_vline = -48

#set the spectral resolution of the spectra in km/s
deltaV = 0.4

#################################################

#JCMT efficiency at 230 GHz to convert TA* to Tmb
eta = 0.6

# retrieving the source data and information
datfile = fits.open(path+'/'+filename)
cube = SpectralCube.read(datfile)
wcs = WCS(datfile[0].header) # vel and position info from WCS in header

# extract the world coordinates of all the pixels using the world property, 
# which returns the spectral axis then the two positional coordinates in reverse order 
# (in the same order as the data indices). 
velo, lat, long  = cube.world[:] 

# convert the velocity units from m/s (in the fits header) to km/s
cube2 = cube.with_spectral_unit(u.km / u.s)
        
# initialize the arrays to write to an ascii table file at the end
x = []
y = []
glat = []
glong = []
vlsr = []
TA = []
sigma = []
fwhm = []
TAdv = []
Tmb = []
Tmbdv = []

# sets an index for the array entries
n = 0

# Loop over all pixels in the x and y ranges provided, extract the spectrum at that pixel,
# fit a gaussian to the spectrum and print and plot the results
for i in range(min_xpix,max_xpix+1):
    for j in range(min_ypix,max_ypix+1):
        # grab a spectrum at pixel x, y 
        x.append(i)
        y.append(j)
        T = cube2[:, j, i]               # get the intensity/temperature along the spectral axis
        vel = cube2.spectral_axis        # set the velocity axis
        glat.append(lat[0,j,0])          # get the DEC/lat of pixel x/y from the world coordinates
        glong.append(long[0,0,i])        # get the RA/long of pixel x/y from the world coordinates

        # Fit the data using a Gaussian
        g_init = models.Gaussian1D(amplitude=1.0, mean=vel_guess, stddev=1.0)
        fit_g = fitting.LevMarLSQFitter()
        g = fit_g(g_init, vel, T)
        #print(g)
        
        gT = g.amplitude.value   # Line peak temperature of the gaussian fit  (K)
        gV = g.mean.value        # line centre velocity of the gaussian fit (km/s)
        gS = g.stddev.value      # standard deviation (sigma) width of the gaussian fit (km/s)
        
        noise = T.std()                      # Calculate the rms noise in the spectrum
        intnoise = deltaV * noise * u.km / u.s  # multiple the noise by the channel width we can calculate the SNR using the intergrated intensity/moment 0
        newgT = gT * u.K                     # give the Gaussian fit temp the units of K                   
        #print('noise = ',noise)
        #print('integrated noise = ', intnoise)
        #print('max = ',max)
              
        # calculate the integrated intensity of the spectrum over the line
        line = cube2.spectral_slab(min_vline*u.km / u.s, max_vline*u.km / u.s)
        max = line[:,j,i].max()       # calculate the maximum channel height in the region covered by the line
        mom0 = line.moment(order=0)
        mom_line = abs(mom0[j,i])     # extract the moment 0 value fo the desire x, y position/pixel
        #print('moment line =',mom_line)
  
        # calculate the SNR 
        snr = mom_line/intnoise
        #print('SNR = ',snr)
        
        # If the SNR is > some value, the max channel is >  something * noise
        # and the FWHM is > some minimum width and < some maximum width
        # then accept the values of the gaussian fits
        # if not, then we assume that the spectrum is weak, the fit is nonsense and so
        # we set the values to 0 so we know what to ignore in the table
        if snr > snr_min and max > (snr_min*noise) and (2.355*gS) > dv_min and (2.355*gS) < dv_max :
            TA.append(gT)               # Line peak temperature of the gaussian fit  (K)
            vlsr.append(gV)             # line centre velocity of the gaussian fit (km/s)
            sigma.append(gS)            # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(2.355 * gS)     # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(gT/eta)          # Calibrated Line peak temperature of the gaussian fit  (K)
        else:
            TA.append(0)           # Line peak temperature of the gaussian fit  (K)
            vlsr.append(0)         # line centre velocity of the gaussian fit (km/s)
            sigma.append(0)        # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(0)         # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(0)          # Calibrated Line peak temperature of the gaussian fit  (K)
        
        # print the fit results to the screen
        #print(x[n], y[n], glat[n], glong[n], "%5.2f"%TA[n], "%5.2f"%Tmb[n], "%5.2f"%vlsr[n],"%5.2f"%sigma[n], "%5.2f"%fwhm[n])

        
        n += 1    #increment the array index

# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = x
out['Pix_y'] = y
out['GLat(deg)'] = glat
out['GLat(deg)'].info.format = '8.7f'
out['GLong(deg)'] = glong 
out['GLong(deg)'].info.format = '8.7f'
out['TA(K)'] = TA
out['TA(K)'].info.format = '6.2f'
out['Tmb(K)'] = Tmb
out['Tmb(K)'].info.format = '6.2f'
out['VLSR(km/s)'] = vlsr
out['VLSR(km/s)'].info.format = '6.2f'
out['sigma(km/s)'] = sigma
out['sigma(km/s)'].info.format = '6.2f'
out['FWHM(km/s)'] = fwhm
out['FWHM(km/s)'].info.format = '6.2f'
#out['TAdv(K km/s)'] = TAdv
#out['TAdv(K km/s)'].info.format = '6.2f'
#out['Tmbdv(K km/s)'] = Tmbdv
#out['Tmbdv(K km/s)'].info.format = '6.2f'

# Write the gaussian fits of all pixels to a text file 
ascii.write(out, path+'/'+outfile,  overwrite=True, format='tab')

# Turn warnings back on
warnings.filterwarnings('default')

print('done')

done


#### The data from the gauss. fits are now stored. Let's open that file and put the x, y, glong, glat, Tmb and FWHM into lists. 

In [3]:
# code to get FWHM and Tmb for all pixels

# importing pandas module 
import pandas as pd 
  
# importing regex module
import re

warnings.filterwarnings('ignore') # Turn warnings off (because they're annoyong!)

path='/Users/jane/Desktop'   # path to file

filename='s138_HCO+_gaussfits.txt'       # input fits file name
outfile = 'radexHCO+.inp' # file name of the radex file
X = pd.read_csv('/Users/jane/Desktop/s138_HCO+_gaussfits.txt', sep="\t", header=None)

df = pd.DataFrame(data=X)

# put pixel x, y, glat, glong, Tmb, FWHM and VLSR in lists and remove headers

xPix = np.array(df[0].tolist())
xPix = xPix[1:]
yPix = np.array(df[1].tolist())
yPix = yPix[1:]
glat = np.array(df[2].tolist())
glat = glat[1:]
glong = np.array(df[3].tolist())
glong = glong[1:]
tmb = (df[5].tolist())
tmb = tmb[1:]
fwhm = np.array(df[8].tolist())
fwhm = fwhm[1:]
vlsr = np.array(df[6].tolist())
vlsr = vlsr[1:]


print('done')

done


#### Store all Tkin from temp file

In [4]:
filename = '/Users/jane/Desktop/s138_temp_regrid.fits'
outfile = '/Users/jane/Desktop/s138_temp_regrid.txt'

# load file, read data, wcs, and header
datfile = fits.open(filename)
dat = datfile[0].data
wcs = WCS(datfile[0].header)

#  y (vert) = 1st position (starting at bottom), x (hor) = 2nd position (starting from left)
hor = []
vert = []
I = []
glatF = []
glongF = []

i = 0
j = 0

# scan through y-axis
for j in range(dat.shape[0]):
    # scan through x-axis 
    for i in range(dat.shape[1]):
        # set the value of the pixel
        zpix = dat[j,i]  # the flux value is found at pixel x, y 
        hor.append(i)
        vert.append(j)
        I.append(zpix)
        gxF, gyF = wcs.wcs_pix2world(i , j , 0) # convert the pixel numbers to WCS coordinates
        glongF.append(gxF)
        glatF.append(gyF)
        

# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = hor
out['Pix_y'] = vert
out['GLat(deg)'] = glatF
out['GLat(deg)'].info.format = '8.5f'
out['GLong(deg)'] = glongF
out['GLong(deg)'].info.format = '8.5f'
out['T(K)'] = I
out['T(K)'].info.format = '6.2f'

# Write the values of all pixels to a text file 
ascii.write(out, outfile,  overwrite=True, format='tab')

print('done')

done


#### Keep only pixels with gauss. fit data and check glat and glong match for these pixels

In [5]:
filename='/Users/jane/Desktop/s138_temp_regrid.txt'      
M = pd.read_csv('/Users/jane/Desktop/s138_temp_regrid.txt', sep="\t", header=None)

df = pd.DataFrame(data=M)

# put pixel Tkin in list and remove header (T denotes full list of all pixels)
xPixT = np.array(df[0].tolist())
xPixT = xPixT[1:]
yPixT = np.array(df[1].tolist())
yPixT = yPixT[1:]
tkT = (df[4].tolist())
tkT = tkF[1:]

# only keep pixels that we have tmb and fwhm data for

tk, xx, yy= [], [], []

#print(xPixF, yPixF, xPix, yPix)

for i in range(len(xPix)):
    for j in range(len(xPixF)):
        if ((xPix[i] == xPixF[j]) and (yPix[i] == yPixF[j])):
            tk.append(tkF[j])
            xx.append(xPixF[j])
            yy.append(yPixF[j])
            
if (np.all(xPix != xx) or np.all(yPix != yy)):
    print('error')
    
print('done')

done


#### Run results above through radex for each non-zero pixel

In [6]:
#creating radex file with FWHM and Tmb obtained above


import math
import os
import sys
#
# Run a series of Radex models to retrieve the column density
#
# Author: Floris van der Tak , version:  06oct08 
# This version by R. Plume - May 2022

maxiter = 100
debug   = False

# lists for all new data (omitting null rows)
xPixFull, yPixFull, glongFull, glatFull, colDen, tkFull, tmbFull, vlsrFull, fwhmFull = [], [], [], [], [], [], [], [], []

# loop through all pixels

ran = len(xPix)

for i in range(ran):
    
    #only run non-zero rows
    if tmb[i] != '0.00':

        #converting values to floats 
        b = np.asarray(tmb[i], dtype=float)
        c = np.asarray(fwhm[i], dtype=float)
        d = np.asarray(tk[i], dtype=float)

        mole = 'HCO+'      #molecule- CAN CHANGE THIS
        freq = 267.6     # frequency GHz
        tkin = d      # Tkin (K)
        nh2 = 1.0e5      # nH2 cm^-3
        tbg = 2.73       # Tbg (K)
        obs = b          # Observed line intensity (K)- Tmb
        dv = c           # FWHM line width km/s
        bw = 0.01       # Bandwidth (GHz)
        tol = 0.01       # tolerance

        radexpath = '/Users/jane/Desktop/Radex/data/'
        extension = '.dat'

        #make sure the file exists (else Radex bonks)
        #if (os.path.exists(radexpath+mole+extension)):
            #print("Using data file",radexpath+mole+extension)
        #else:
            #print("Cannot find file: ",radexpath+mole+extension)
            #sys.exit()

        def write_input(cdmol):
            file = open('/Users/jane/Desktop/radexHCO+.inp','w')
            file.write(mole+'.dat\n') 
            file.write('/Users/jane/Desktop/radexHCO+.out\n')
            file.write(str(freq*(1-bw))+' '+str(freq/(1-bw))+'\n')
            file.write(str(tkin)+'\n')
            file.write('1\n')
            file.write('H2\n')
            file.write(str(nh2)+'\n')
            file.write(str(tbg)+'\n')
            file.write(str(cdmol)+'\n')
            file.write(str(dv)+'\n')
            file.write('0\n')
            file.close()

        def read_radex():
            file  = open('/Users/jane/Desktop/radexHCO+.out')
            lines = file.readlines()
            file.close()
            if (lines[-2].split()[-1] != '(erg/cm2/s)'):
                print("Error: Ambiguous line selection. Reduce bandwidth?")
                print("See radex.out for details")
                sys.exit()
            return float(lines[-1].split()[-2])

        # Begin of main program
        oflx = obs*dv
        eps  = 1.0e-20
        iter = 0

        # Starting values of column density and fit residual
        cdmol = 1e12
        ratio = 0

        while (ratio > (1+tol)) or (ratio < (1-tol)) :
            iter += 1
            write_input(cdmol)
            os.system('/Users/jane/Desktop/Radex/bin/radex < /Users/jane/Desktop/radexHCO+.inp > /dev/null')
            mflx  = read_radex()
            if (mflx < eps):
                print("Error: Zero or negative line intensity")
                print("See radex.out for details")
                sys.exit()
            if (debug):
                print("mflx= ",mflx)
            ratio = oflx/mflx
            cdmol = cdmol * ratio
            if (iter > maxiter):
                print("Maximum number of iterations exceeded")
                ratio = 1

        #fmt = "CD %7.2e cm^-2"
        colDen.append(cdmol)
        xPixFull.append(xPix[i])
        yPixFull.append(yPix[i])
        glongFull.append(glong[i])
        glatFull.append(glat[i])
        tkFull.append(tk[i])
        tmbFull.append(tmb[i])
        vlsrFull.append(vlsr[i])
        fwhmFull.append(fwhm[i])
        
print('done')

done


In [7]:
#writing results to table 

path='/Users/jane/Desktop'   # path to file
outfile = 'columnDenHCO+.txt' # file name of the gaussian fit results

# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = xPixFull
out['Pix_y'] = yPixFull
out['GLat(deg)'] = glatFull
#out['GLat(deg)'].info.format = '8.7f'
out['GLong(deg)'] = glongFull
out['Tk(K)'] = tkFull
out['Tmb(K)'] = tmbFull
#out['Tmb(K)'].info.format = '6.2f'
out['VLSR(km/s)'] = vlsrFull
#out['VLSR(km/s)'].info.format = '6.2f'
out['FWHM(km/s)'] = fwhmFull
#out['FWHM(km/s)'].info.format = '6.2f'
#out['GLong(deg)'].info.format = '8.7f'
out['Col. Den. (cm^-2)'] = colDen
#out['Col. Den. (cm^-2)'].info.format = '6.2f'

# Write the gaussian fits of all pixels to a text file 
ascii.write(out, path+'/'+outfile,  overwrite=True, format='tab')

print('done')

done


### txt file to excel

In [8]:
import pandas as pd
X = pd.read_csv('/Users/jane/Desktop/columnDenHCO+.txt', sep="\t", header=None)

df = pd.DataFrame(data=X)

df.to_excel('/Users/jane/Desktop/colDenExcelHCO+.xlsx')

print('done')

done
